# Formation PyTorch : les bases pour être autonome 
#### 3 novembre 2022 de 9h à 17h à l'OMP (salle Coriolis)

# Partie 1
## Manipuler les objets basiques de PyTorch : tenseurs, paramètres, modèles... 

Lien vers le GG Colab : https://colab.research.google.com/drive/1EVP2ksGC8V9vc9kbhHKXMJey9i6sobF_?usp=sharing

Dans cette première partie, nous allons : 
 * Découvrir et manipuler les éléments de base de PyTorch,
 * Découvrir les bases de l'optimisation avec PyTorch.


In [ ]:
import torch

### Les tenseurs

Le tenseur est l'objet incontournable de PyTorch, l'équivalent du array pour la librairie numpy. La très grande majorité des opérations effectuées avec PyTorch sont effectuées sur des tenseurs. On va voir ci-dessous une liste non exhaustive des opérations possibles sur les tenseurs.

https://pytorch.org/docs/stable/tensors.html?highlight=torch+tensor#torch.Tensor

In [ ]:
X = torch.tensor([
    [2, 1.5, 3],
    [4, 2, 4.5]
])

In [ ]:
X.shape

In [ ]:
X.T

In [ ]:
X.sum()

In [ ]:
X.sum().item()

In [ ]:
torch.sum(X, dim=0)

In [ ]:
X.unsqueeze(2).shape

In [ ]:
X.view(-1)

In [ ]:
X.repeat(1, 2)

In [ ]:
torch.ones((2,2))

In [ ]:
torch.randn((2,2))

In [ ]:
X.to('cpu')

### Paramètres et rétro-propagation

Les paramètres sont des objets dont les valeurs ont vocation à être optimisées, comme par exemple les paramètres d'un modèle linéaire pour faire une régression.

https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html?highlight=parameter#torch.nn.parameter.Parameter

In [ ]:
from torch.nn.parameter import Parameter

X = Parameter(torch.tensor([[0., 1.], [2.,3.]]), requires_grad=True)

In [ ]:
X

In [ ]:
X.data

In [ ]:
X.requires_grad

In [ ]:
print(X.grad)

In [ ]:
print(X.grad_fn)

In [ ]:
Y = torch.mm(X, X.T)
loss = torch.sum(Y)

In [ ]:
Y.grad_fn

In [ ]:
loss.grad_fn

<img src="backpropagation.png" width=500 height=200 />

Source : Sun, Yubiao & Sun, Qiankun & Qin, Kan. (2021). Physics-Based Deep Learning for Flow Problems. Energies. 14. 7760. 10.3390/en14227760. 

In [ ]:
loss

In [ ]:
loss.backward()

In [ ]:
X.grad

In [ ]:
print(Y.grad)

In [ ]:
X = Parameter(torch.tensor([[0., 1.], [2.,3.]]), requires_grad=True)
Y = X**2
Y.retain_grad()
loss = torch.sum(Y)
loss.backward()

In [ ]:
Y.grad

In [ ]:
X = Parameter(torch.tensor([[0., 1.], [2.,3.]]), requires_grad=True)
Y = torch.mm(X, X.T)
Y.backward()
X.grad

In [ ]:
X = Parameter(torch.tensor([[0., 1.], [2.,3.]]), requires_grad=True)
Y = torch.mm(X, X.T)
g = torch.ones_like(X)
Y.backward(g)
X.grad

Comment calculer $ \Large{\frac{\partial Y_{11}}{\partial X}} $ ?

In [ ]:
# A compléter

### Optimisation des paramètres 

On va voir ci-dessous quels outils utiliser pour l'optimisation des paramètres.

https://pytorch.org/docs/stable/optim.html?highlight=optimizer#torch.optim.Optimizer

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
X = torch.linspace(0, 10, 100).unsqueeze(1)
Y = 2.5 + 3*X + 3*torch.randn((100,1))

fig = plt.figure()
plt.scatter(X, Y)
plt.show()

In [ ]:
import torch.nn.functional as F 

# Complétez la définition de a, le coefficient directeur, et b, l'ordonnée à l'origine
a = ...
b = ...

# Complétez la définition de l'optimiseur : 
# https://pytorch.org/docs/stable/generated/torch.optim.SGD.html?highlight=sgd#torch.optim.SGD
optimizer = torch.optim.SGD(..., lr=1e-2)

for epoch in range(10):
    # Complétez le calcul de y_pred
    y_pred = ...
    mse = F.mse_loss(Y, y_pred)
    mse.backward() # On calcule les gradients de mse par rapport à a et b 
    optimizer.step() # On met à jour les valeurs de a et b 
    optimizer.zero_grad() # On 'remet' les gradients à zéro pour la prochaine epoch 
    print(f'=== Epoch {epoch} ===  ')
    print(f'MSE: {mse.item():.3f}')
    

In [ ]:
with torch.no_grad():
    y_pred = ...
    
fig = plt.figure()
plt.scatter(X, Y)
plt.plot(X, y_pred, color='red')
plt.show()

### Les modèles

Les modèles sont les objets de PyTorch qui définissent les réseaux de neurones.

https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module

In [ ]:
class LinearModel(torch.nn.Module):
    def __init__(self, x_dim):
        super(LinearModel, self).__init__()
        self.x_dim = x_dim
        self.linear = torch.nn.Linear(x_dim, 1)
        
    def forward(self, x):
        return self.linear(x)

In [ ]:
linear_model = LinearModel(X.shape[1])
linear_model

In [ ]:
for param in linear_model.named_parameters():
    print(param[0])

In [ ]:
linear_model.linear.weight

In [ ]:
linear_model.linear.weight.shape

In [ ]:
linear_model.linear.bias

In [ ]:
optimizer = torch.optim.SGD(linear_model.parameters(), lr=1e-2)

for epoch in range(10):
    y_pred = linear_model(X)
    mse = F.mse_loss(Y, y_pred)
    mse.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f'=== Epoch {epoch} ===  ')
    print(f'MSE: {mse.item():.3f}')

In [ ]:
with torch.no_grad():
    y_pred = linear_model(X)
    
fig = plt.figure()
plt.scatter(X, Y)
plt.plot(X, y_pred, color='red')
plt.show()

### Data Loaders

Les data loader sont des objets de PyTorch pour charger les données de manière à faciliter l'optimisation par "Batch stochastic gradient descent".

https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader

In [ ]:
X = torch.linspace(0, 10, 100).unsqueeze(1)
Y = 2.5 + 3*X + 3*torch.randn((100,1))

data = torch.utils.data.TensorDataset(X, Y)

In [ ]:
data

In [ ]:
len(data)

In [ ]:
loader = torch.utils.data.DataLoader(data, batch_size=20, shuffle=True)

In [ ]:
len(loader)

In [ ]:
optimizer = torch.optim.SGD(linear_model.parameters(), lr=1e-2)

for epoch in range(10):
    for batch, y in loader:
        ...
    print(f'=== Epoch {epoch} ===  ')
    print(f'MSE: {mse.item():.3f}')